In [219]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import random
import os

In [221]:
# Reading data files from my computer

def get_data():
    directory_pwd = '/Users/charbel/Desktop/GitHub/COVID-Hackathon/JohnHopkinsDataSet/'
    directory = os.fsencode(directory_pwd)
    data = []
    labels = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            full_path = directory_pwd + filename
            df = pd.read_csv(full_path)
            df = df[df.Province_State != "Recovered"]
            df = df.drop(["Lat", "Long_", "Last_Update", "Province_State", "Country_Region", "UID", "ISO3", "FIPS", "Deaths", "Active"], 1)
            df = df.fillna(df.median())
            labels.append(df['Confirmed'].to_numpy())
            df = df.drop(['Confirmed'], 1)
            data.append(df.to_numpy())
    return data, labels

data, labels = get_data()
data = np.stack(data)
print("data", data.shape)
labels = np.stack(labels)
print("labels", labels.shape)
# (# of days, #of states/province, #of features)
        

data (104, 58, 7)
labels (104, 58)


In [222]:
class RNN(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.rnn = nn.GRU(input_size=58*7,
                          hidden_size=hidden_size,
                          batch_first=True)
        self.proj = nn.Linear(in_features=hidden_size,
                              out_features=58)
    
    def forward(self, inp):
        X = inp.reshape([-1,58*7]).unsqueeze(dim=0)
        out, last_hidden = self.rnn(X)
        out_seq = self.proj(out)
        return out_seq.squeeze()

In [223]:
rnn = RNN(406)
y = rnn(torch.Tensor(data))
y.shape

torch.Size([104, 58])

In [224]:
y

tensor([[ 0.6664,  0.2482,  0.7946,  ...,  0.5328, -0.3275, -0.1726],
        [ 0.6664,  0.2482,  0.7946,  ...,  0.5328, -0.3275, -0.1726],
        [ 0.6032,  0.1966,  0.5804,  ...,  0.4549, -0.1044, -0.3602],
        ...,
        [ 0.3819,  0.1311,  0.4628,  ...,  0.8458,  0.2055, -0.0764],
        [ 0.2310,  0.1157,  0.4124,  ...,  0.5201,  0.2668,  0.1190],
        [ 0.2310,  0.1157,  0.4124,  ...,  0.5201,  0.2668,  0.1190]],
       grad_fn=<SqueezeBackward0>)